In [140]:
from pyqubo import Binary, Array, Num
import numpy as np
from dimod import ExactSolver
import neal
from gen import gen_average_degree_directed, gen_planted_path, StandardGraph
from typing import List, Tuple, Callable
from math import log2, floor

In [141]:
def make_tail_graph(graph : StandardGraph, tail_length : int) -> StandardGraph:

	if tail_length == 0:
		tail_graph = StandardGraph(0, [])
		tail_graph.vertices += graph.vertices
		tail_graph.edges = graph.edges.copy()
		return tail_graph

	tail_graph = StandardGraph(0, [])
	tail_graph.vertices += graph.vertices + tail_length
	tail_graph.edges = graph.edges.copy()
	
	for i in range(0,graph.vertices):
		tail_graph.edges.append((i,graph.vertices))
	
	for i in range(graph.vertices , graph.vertices + tail_length - 1):
		tail_graph.edges.append((i, i+1))

	return tail_graph


def make_vertex_edges(graph : StandardGraph) -> List[List[Tuple[int, int, int]]] :
	out = [[] for i in range(graph.vertices)]

	for i in range(len(graph.edges)):
		(v_1, v_2) = graph.edges[i]
		
		out[v_1].append((i,v_1,v_2))
		out[v_2].append((i,v_1,v_2))
	
	return out

In [142]:
graph = StandardGraph(5, [
	(0,1),
    (1,2),
    (2,3),
    (3,4),
])

# graph = gen_average_degree_directed(6, 1)

# max length
N = graph.vertices
K = N - 1

P = N**2

binaries = range(floor(log2(K)) + 1)

tail_graph = make_tail_graph(graph, K)

vertex_edges = make_vertex_edges(graph)

In [143]:


vertex_vars = Array.create('vertex', tail_graph.vertices, vartype='BINARY')
edge_vars = Array.create('edge', len(tail_graph.edges), vartype='BINARY')
initial_edge_vars = Array.create('init_edge', tail_graph.vertices, vartype='BINARY')
terminal_edge_vars = Array.create('term_edge', tail_graph.vertices, vartype='BINARY') 

flow_vars = Array.create('flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY')
flow_vars = Array.create('contra_flow', shape=(len(binaries), len(tail_graph.edges)), vartype='BINARY')


candy_exp = sum(edge_vars)

initial_node_exp = P * (1 - sum(initial_edge_vars[i] for i in range(graph.vertices)))**2

terminal_node_exp = P * (1 - sum(terminal_edge_vars[i] for i in range(graph.vertices)))**2

all_node_deg_2_exp = 0
for i in range(graph.vertices):
	all_node_deg_2_exp += P * (2 * vertex_vars[i] - sum(vertex_edges[i]))**2






TypeError: unsupported operand type(s) for +: 'int' and 'tuple'